In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SCAN (https://scan-interfax.ru/) - это система Международной информационной группы "Интерфакс", предназначенная для комплексного решения задач в сфере управления репутацией и анализа эффективности PR. Решает задачи по мониторингу СМИ и соцсетей, анализу медиа-поля, проверке деловой репутации компаний и персон, позволяет пользователям оперативно реагировать на появление негатива в СМИ и соцмедиа, путём осуществления автоматизированного сбора и анализа публичной информации из более чем 60 тысяч источников.

Одной из функциональных возможностей SCAN является фактографический анализ новостной информации - выделение контекстов, в которых упоминается событие или действие некоторого субъекта на заданную тематику и с заданной тональностью.
На текущий момент SCAN умеет определять контексты более чем по 800 различным темам. Разработка каждой темы представляет собой длительный процесс по предварительному сбору и анализу большого корпуса текстовой информации для выделения характерных фраз, с последующим написанием машинных правил на специальном DSL (domain-specific language), в котором задействован целый отдел прикладных лингвистов.

Мы предлагаем вам принять участие в решении значимой для проекта SCAN задачи, которая позволяет расширить спектр выделяемых системой контекстов и снизит нагрузку на лингвистов:

Разработка средства автоматизированного поиска контекстов на заданные тематики. Нам важна семантическая близость к уже проработанным нами контекстам, чтобы не требовалось описывать каждый контекст в рамках одной тематики машинными правилами на специальном DSL:

В качестве исходных данных выступают наборы размеченных корпусов на различные тематики.
В качестве искомого контекста может выступать как часть предложения исходного текста, так и целое предложение, или даже набор предложений на ту же тему.

In [5]:
!pip install pymorphy2

In [6]:
import numpy as np
import pandas as pd
import re

import datatable as dt
from dask import dataframe as dd 

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
from nltk.stem import WordNetLemmatizer
import pymorphy2
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import transformers
import torch
import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

## Logistic Regression

## Применение классификации текста с использованием логистической регрессии

In [7]:
file = '../input/scan-classification-challange/df_train.csv'
df_dt = dt.fread(file)
df_dt = df_dt.to_pandas()
df_dt.sample(3)

In [8]:
train = pd.read_csv('../input/scan-classification-challange/df_train.csv')
train.sample(5)

In [9]:
# Посмотрим, что из себя представляет значение с текстом

train['text'][12]

In [14]:
# Удаляем дубликаты

train.drop_duplicates(subset={'text'}, inplace=True) 
train.shape

In [15]:
train['class'].value_counts()

In [16]:
# Видно, что данные сильно не сбалансированны

from collections import Counter
counter = Counter(train['class'])
plt.figure(figsize=(15, 6))
plt.bar(counter.keys(), counter.values(), width=2)
plt.xticks(rotation=90)
plt.show()

In [17]:
test = pd.read_csv('../input/scan-classification-challange/df_test.csv', index_col=0)
test.sample(3)

In [18]:
sample_submission = pd.read_csv('../input/scan-classification-challange/sample_submission.csv')

In [19]:
# Названия категорий переводим в числовой формат и записываем в отдельный столбец 

train['encoded_cat'] = train['class'].astype('category').cat.codes
train.sample(5)

In [22]:
train[['class', 'encoded_cat']].value_counts()

In [23]:
# Создадим функцию по очистке данных. Будем переводить слова в нижний регистр, 
# удалять стоп слова, удалять числа и раздичные знаки которые не несут смысловой нагрузки. 
# Все слова преобразуем к их первоначалоной форме (Лемматизация)

morph = pymorphy2.MorphAnalyzer()
patterns = "[A-Z|a-z|0-9!#$%&'()*+,./:“″;”<=>?@[\]^_`{|}~—\"\-•–«»]+"
stops = set(stopwords.words("russian"))
def clean(text):
    text = text.lower()
    text = re.sub(patterns, ' ', text)
    tokens = []
    for token in text.split():
        if token and token not in stops:
            token = token.strip()
            token = morph.normal_forms(token)[0]  # Лемматизация
            #token = stemmer.stem(token) # Стеммизация
            tokens.append(token)
    return ' '.join(tokens)

In [24]:
# Применим функцию очистки к train
train['clean'] = train['text'].apply(lambda x: clean(x))
train[['clean', 'text']]

In [25]:
# Применим функцию очистки также для тестовых данных

test['clean'] = test['text'].apply(lambda x: clean(x))

In [26]:
# Выделим X, y. X - это будет, наш обработанный текст, y -  наш класс

y = train.encoded_cat.values
X = train.drop(['encoded_cat', 'text'], axis=1)

In [27]:
## Разделим все данные на train test в соотношении 80/20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y) # stratify=y

## Преобразуем наши текстовые данны в токены.

In [28]:
## CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

X_train_review_bow = vect.fit_transform(X_train['clean'])
X_test_review_bow = vect.transform(X_test['clean'])
X_sub_rev_bow = vect.transform(test['clean'])
#X_train_review_bow = vect.fit_transform(X_train)
#X_test_review_bow = vect.transform(X_test)

print('X_train_review_bow shape: ', X_train_review_bow.shape)
print('X_test_review_bow shape: ', X_test_review_bow.shape)
print('X_sub_review_bow shape: ', X_sub_rev_bow.shape)

## Tf-Idf
Tf-Idf расшифровывается как частота термина, обратная частоте документа, и вместо вычисления количества каждого слова в каждом документе набора данных (Bow) он вычисляет нормализованное количество, где каждое количество слов делится на количество документов, в которых встречается это слово. .

Tf-idf(w, d)= Bow(w, d) * log(Общее количество документов/(Количество документов, в которых встречается слово w))

Если слово часто встречается в определенном документе, но не во многих других документах, наиболее вероятно, что это слово имеет особое значение для этого документа и получает большее количество, чем раньше, благодаря высокому Idf. С другой стороны, если слово появляется во многих документах, то его Idf близок к 1, а логарифм превращает 1 в 0 и уменьшает его влияние.

In [29]:
## Tf-Idf

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_review_tfidf = vectorizer.fit_transform(X_train['clean'])
X_test_review_tfidf = vectorizer.transform(X_test['clean'])
X_sub_review_tfidf = vectorizer.transform(test['clean'])

print('X_train_review_tfidf shape: ', X_train_review_tfidf.shape)
print('X_test_review_tfidf shape: ', X_test_review_tfidf.shape)
print('X_sub_review_tfidf shape: ', X_sub_review_tfidf.shape)

## Логистическая регрессия
После создания 80/20 разделения набора данных на поезд-тест я применил логистическую регрессию, которая представляет собой алгоритм классификации, используемый для решения задач бинарной классификации. Классификатор логистической регрессии использует взвешенную комбинацию входных признаков и пропускает их через сигмовидную функцию. Сигмовидная функция преобразует любое введенное вещественное число в число от 0 до 1.

In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
y_test_arg=np.argmax(y_test)
clf = MultinomialNB()
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow) #prediction from model
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred, average = 'micro'))
print('Precision: ', precision_score(y_test, y_pred, average = 'weighted', zero_division = 1))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [31]:
clf = MultinomialNB()
y_test_arg=np.argmax(y_test)
clf.fit(X_train_review_tfidf, y_train)

y_pred = clf.predict(X_test_review_tfidf)
#y_pred = clf.predict(X_sub_review_tfidf)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [32]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42)
clf.fit(X_train_review_tfidf, y_train)

#y_pred = clf.predict(X_test_review_tfidf)
y_pred = clf.predict(X_test_review_tfidf)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [33]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

## SMOTE

In [ ]:
""""from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
 

# evaluate a model
def evaluate_model(X, y, model):
# define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=1)
# evaluate model
    scores = cross_val_score(model, X_train_review_tfidf, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
# load the dataset
#X, y = load_dataset(full_path)
# define the model
model = RandomForestClassifier(n_estimators=100, class_weight='balanced') #`1000
# evaluate the model
scores = evaluate_model(X_train_review_tfidf, y_train, model)
# summarize performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))
"""

## Ngramm

In [ ]:
# CountVectorizer
vect = CountVectorizer(ngram_range=(1, 2))

X_train_review_bow = vect.fit_transform(X_train['clean'])
X_test_review_bow = vect.transform(X_test['clean'])
X_sub_rev_bow = vect.transform(test['clean'])

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', C = 10, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# HashingVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer = HashingVectorizer(ngram_range=(1, 3))

X_train_review_hv = vectorizer.fit_transform(X_train['clean'])
X_test_review_hv = vectorizer.transform(X_test['clean'])
X_sub_review_hv = vectorizer.transform(test['clean'])

In [ ]:
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', random_state=42)
clf.fit(X_train_review_hv, y_train)

y_pred = clf.predict(X_test_review_hv)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))

## Попробуем другие модели из sklearn

In [34]:
# C-классификация опорных векторов

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_review_bow, y_train)
y_pred_svc = svclassifier.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred_svc))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [35]:
# Классификатор, реализующий голосование k ближайших соседей

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(X_train_review_bow, y_train)
y_pred = knn.predict(X_test_review_bow)

print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

## Гиперпараметры

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class='multinomial', penalty='l2', random_state=42)
#penalty = ['l1','l2']
solver = ['newton-cg', 'sag', 'saga', 'lbfgs']
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
grid = dict(alpha=alpha)
c_values = [100, 10, 1.0]
#param_grid = {'C':[1, 10]}
param_grid = dict(C=c_values, solver=solver)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, verbose=10, cv=cv, scoring='accuracy', error_score=0)
grid_search.fit(X_train_review_bow, y_train)
#print("Best: %f using %s" % (grid_search.best_params_))

### Попробуем обучить с полученными гипер параметрами 

In [36]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='newton-cg', penalty='l2', C = 10, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

In [37]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial', solver='saga', penalty='l2', C = 100, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_test_review_bow)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))
print('Test F1: ', f1_score(y_test, y_pred, average='weighted'))

 С подобранными гиперпараметрами получилось улучшить модель до 0.897

### делаем предсказание на Test и выкладываем на Kaggle

In [ ]:
clf = MultinomialNB(alpha=1)
clf.fit(X_train_review_tfidf, y_train)

y_pred = clf.predict(X_sub_review_tfidf)
#print('Test Accuracy: ', accuracy_score(y_test, y_pred))

In [ ]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_review_bow, y_train)
y_pred_svc = svclassifier.predict(X_sub_review_tfidf)

In [ ]:
clf = LogisticRegression(random_state=42)
clf.fit(X_train_review_tfidf, y_train)

#y_pred = clf.predict(X_test_review_tfidf)
y_pred = clf.predict(X_sub_review_tfidf)
#print('Test Accuracy: ', accuracy_score(y_test, y_pred))

In [ ]:
clf = LogisticRegression(multi_class='multinomial', solver='saga', penalty='l2', C = 100, random_state=42)
clf.fit(X_train_review_bow, y_train)

y_pred = clf.predict(X_sub_rev_bow)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train['class'].unique())
#list(le.classes_)
num = le.transform(train['class'].unique())


In [ ]:
y_sub = list(le.inverse_transform(y_pred))

In [ ]:
#sub_predict = model.predict(X_sub)
sample_submission['class'] = y_sub
sample_submission.to_csv('LR_submission.csv', index=False)

# Bert

## Fine Tuning DistilBERT TensorFlow

In [30]:
#!pip uninstall transformers --yes
#!pip uninstall h5py

!pip install transformers
#!pip install h5py==2.10.0
!pip install pytorch
!pip install pytorch-transformers

In [40]:
import transformers
from transformers import DistilBertTokenizer
from transformers import DistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
#import pandas as pd
import json
import gc
#import numpy as np

In [41]:
# Токенизируем текст 
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_encodings = tokenizer(X_train['clean'].to_list(), truncation=True, padding=True)
test_encodings = tokenizer(test['text'].to_list(), truncation=True, padding=True)

X_test_encodings = tokenizer(X_test['clean'].to_list(), truncation=True, padding=True)

In [42]:
y_sub = np.zeros(len(test))
y_sub.shape

In [43]:
# Создает dataset с тонекизированными данными

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_encodings), 
                                    list(y_test))) 
val_class = np.zeros(len(test))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    val_class))

In [44]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=50)
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) # 2e-5
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy']) # loss=model.compute_loss

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), 
          epochs=1,
          validation_data=test_dataset.shuffle(1000).batch(16))